<a href="https://colab.research.google.com/github/Lotfullah21/Deep_Learning_MIT-Press/blob/main/Artificial_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artifical Neural Network
in the begining the Anns were thought to work same as brain, that is where the name come from, but in reality it has less to do with brain, even though research is going to make computers to mimic the brain.
but in reality, it is just a mapping function, where it consists of many components like, Input units, Hidden layer, and output layer.



## Input Layer
a layer is where we have bunhch of neurons parallel to each other, this layer can be consisit of one or multiple inputs, an input is also known as a unit in ANNs.
thses units are the primary source of information.


## Hidden Layer
it is the layer between the input and output units,it is where the magic happens, adding non_linearity to our function to map the complex function.

in theorems of general approximation , we are having that a model with a single hidden layer can generalize any kind of function.


# Output Layer
in this layer , after doing certain calculation in hidden layers we output what we have learned and compares it with our desired output

## Network 
a network is consist of many layers, network is also called as model
a model maps the inputs to the outputs by learning meaninful features and representation such that the difference between what the model is givining with what we want should be minimum.


## Loss Function 
this is difference between the model output and its true output(desired feature), we want to minimize this function during training. this says how happy or unhappy we are with our model performance.


## Optimizer
it is a technique which determines how we are going to update our weights based on the loss function.